# Competencia: Clasificación de Péptidos Antimicrobianos

El equipo está conformado por:

    Angie Melissa Calderón Albarracin
    Diego Alejandro Zapata Alcaraz
    Juan David Valencia Quiceno
    Juan Esteban Arroyave Duque

In [195]:
#Importar paquetes
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn import naive_bayes
#from sklearn.feature_selection import RFE
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [171]:
#Cargar la data
data_positiva=pd.read_csv('Datos/DatosPositivos1.csv')
data_negativa=pd.read_csv('Datos/DatosNegativos1.csv')
#data_validacion=pd.read_csv('Datos/DatosValidacion1.csv')

data_positiva['Tipo']='Train'
data_negativa['Tipo']='Train'
data_validacion['Tipo']='Test'

data_pre=pd.concat([data_positiva,data_negativa,data_validacion])
#Nota: La clase 1, corresponde a los péptidos que SON antimicrobianos, 0 los que no.
# Adicional los datos están desbalanceados en la siguiente relacion 60% son positivos 40% negativos.

print('Forma de los datos positivos',data_positiva.shape)
print('Forma de los datos negativos',data_negativa.shape)
print('Forma de los datos validacion',data_validacion.shape)
print('Forma de los datos del preprocesamiento',data_pre.shape)

Forma de los datos positivos (8322, 1762)
Forma de los datos negativos (5561, 1761)
Forma de los datos validacion (1623, 1763)
Forma de los datos del preprocesamiento (15506, 1763)


### Preprocesamiento de datos
    1.Identificación de colummnas adicionales o diferentes entre los datos
    2.Eliminicación de columnas con más 20% de null y nan (Es posible el llenado)
    3.Filtrado de varianza
    4.Correlación entre las variables numéricas

In [169]:
# Encontrar las columnas que no se encuentra en los datasets (positivo, negativo y validación)
col_val=data_validacion.columns
col_pos=data_positiva.columns
col_neg=data_negativa.columns

for i in col_val:
    if i not in col_neg:
        print(i, 'No se encuentra')

Unnamed: 0 No se encuentra
Unnamed: 0.1 No se encuentra


In [173]:
#Elimianción de columnas con mayor cantidad de null o nan
aux=(data_pre.isnull().sum()/len(data_pre))*100
col_total=data_pre.columns
col_remove=[col_total[i] for i in range(len(col_total)) if aux[i]>=20]

data_pre=data_pre.drop(col_remove,axis=1)

In [89]:
#Eliminación de columnas por baja varianza.
#En primera instancia se seleccionan las columnas numéricas, luego se retira la columna 'class' porque es nuestra vble y
a=data_pre.var()
a.drop('class')
col_total=data_pre.columns

col_ana=[col_total[i] for i in range(len(a)) if a[i]<=0.001]
col_ana

In [100]:
data_pre[['charge',
 'taugrant30',
 'QSOgrant1',
 'QSOgrant2']].describe()

,charge,taugrant30,QSOgrant1,QSOgrant2
count,15506.000000,15506.000000,15506.000000,15506.000000
mean,1.869740,0.116402,0.013334,0.014927
std,3.017698,0.426620,0.023763,0.029479
min,-13.894000,0.000000,0.000000,0.000000
25%,-0.004000,0.000000,0.000000,0.000000
50%,1.793000,0.000000,0.005501,0.004394
75%,3.860000,0.000000,0.016044,0.016778
max,21.996000,4.419000,0.524904,0.771977


In [104]:
#cmap = sns.diverging_palette(h_neg=10,h_pos=240,as_cmap=True)
#sns.heatmap(data_pre.corr(), center=0,cmap=cmap, linewidths=1,annot=True, fmt=".2f")
data_pre.corr()


### Estandariazación de los datos

In [174]:
#Volviendo a la estructura original
data_train=data_pre[data_pre['Tipo']=='Train']
data_val=data_pre[data_pre['Tipo']=='Test']

data_train=data_train.drop(['Tipo'],axis=1)
data_test=data_val.drop(['Tipo'],axis=1)

y=data_train.pop('class')
X=data_train

y_val=data_val.pop('class')
X_val=data_val

In [175]:
#Extraer la columnas con valores numéricos
col_num1=[i for i in X_train.columns if "int" in str(X_train[i].dtype) or "float" in str(X_train[i].dtype)]
col_num2=[i for i in X_test.columns if "int" in str(X_test[i].dtype) or "float" in str(X_test[i].dtype)]

#creación del modelo de normalización
scaler=StandardScaler()
X_train_std=scaler.fit_transform(X_train[col_num1])
X_test_std=scaler.transform(X_test[col_num2])

In [176]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Implementación de los modelos de clasificación

Los modelos con los cuales trabajaremos son:
    
    1.LogisticRegressor
    2.RandomForest
    3.Red Neuronal

In [179]:
#Regresión Logistica
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train_std,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [187]:
y_pred = lr.predict(X_test_std)
acc = accuracy_score(y_test, y_pred)
print ("Precisión del clasificador Logitica: %.2f " %(acc*100) )

Precisión del clasificador: 87.36 


In [191]:
# SVM
#Una SVM Lineal
#Una SVM de base Radial y
#Una SVM Polinomial

svm_lineal = SVC(kernel='linear', C=1.0, random_state=0)
svm_rbf = SVC(kernel='rbf', C=1.0, random_state=0)
svm_poly = SVC(kernel='poly', C=1.0, random_state=0)

# Entrenamos los modelos
svm_lineal.fit(X_train_std, y_train)
svm_rbf.fit(X_train_std, y_train)
svm_poly.fit(X_train_std, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [193]:
y_pred = svm_lineal.predict(X_test_std)
acc = accuracy_score(y_test, y_pred)
print ("Precisión del clasificador Lineal: %.2f " %(acc*100.0))

y_pred = svm_rbf.predict(X_test_std)
acc = accuracy_score(y_test, y_pred)
print ("Precisión del clasificador Radial: %.2f " %(acc*100.0))

y_pred = svm_poly.predict(X_test_std)
acc = accuracy_score(y_test, y_pred)
print ("Precisión del clasificador Polinomial: %.2f " %(acc*100.0))

Precisión del clasificador Lineal: 87.18 
Precisión del clasificador Radial: 91.11 
Precisión del clasificador Polinomial: 87.94 


In [196]:
#Naive Bayes
nv = naive_bayes.GaussianNB()
nv.fit(X_train_std,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [197]:
y_pred = nv.predict(X_test_std)
acc = accuracy_score(y_test, y_pred)
print ("Precisión del clasificador Radial: %.2f " %(acc*100.0))

Precisión del clasificador Radial: 75.77 
